# Formulation of the (economic) Dispatch problem
* Packages used:
    * Pyomo
    * ...

#### 1. Parameters
* PV-specific:
    * *None*
* BESS-specific:
    * Max Storage Capacity
    * Charging Capacity == Discharge Capacity
    * Charge-Efficiency
    * Discharge-Efficiency
    * *Roundtrip Efficiency
    * Mac SoC
    * Min SoC
* Other system-specific parameters:
    * Inverter efficiency
    * PV-BESS system cost
    * Max Grid Volume
* Other parameters that the dispatcher requires:
    * Profit {calculated from the power flows and electricity price at each time-step *t*}
    * State-of-Charge [SoC] {traced/calculated from the SoC[*t-1*] and power flows to/from BESS}

#### 2. Decision Variables
*Decision variables in the dispatch model are not the same as in the global optimisation problem. In the dispatch problem, the decision variables are the different power flows at each time-step *t*
* PV_to_Load
* PV_to_BESS
* PV_curtailment
* PV_to_Grid
* BESS_to_Load
* BESS_to_Grid
* Grid_to_Load
* Grid_to_BESS

#### 3. Objective Function
* MinimiseCost = 
    $$
    min( 
        \sum_{t=o}^{8760} (
            C_{grid}^i[t] * P_{Grid-to-Load}[t]                                                     %cost of electricity from grid
            + C_{PV+BESS}^i[t] * [P_{PV-to-Load}[t] + P_{BESS-to-Load}[t]]                          %cost of electricity from PV-BESS system
            + C_{penalty}^{Curtailment} * P_{Curtailment}[t]                                        %penalty for curtailing PV production
            - C_{grid}^i[t] * [[P_{BESS-to-Grid}[t] + P_{PV-to-Grid}[t] ] - P_{Grid-to-BESS}[t]]    %profit from energy arbitrage
        )
    )
    $$

#### 4. Constraints
* Load fulfilment
    $$
        P_{Load}[t] = P_{PV-to-Load}[t] + P_{BESS-to-Load}[t] + P_{Grid}[t]
    $$
* Solar production
    $$
        P_{PV production}[t] = \frac{P_{PV-to-Load}[t] + P_{PV-to-BESS}[t] + P_{PV-to-Grid}[t]}{\xi_{inverter}} + P_{Curtailment}[t]
    $$
* SoC range
    $$
        SoC^{min} < SoC[t] < SoC^{max} ; \qquad \forall \ \ t
    $$
* SoC tracking
    $$
        SoC[t] = (
            SoC[t-1]
            + ( P_{PV-to-BESS}[t] + P_{Grid-to-BESS}[t] * \xi_{charge} ) * \Delta{t}
            - \frac{P_{BESS-to-Load}[t] + P_{BESS-to-Grid}[t]}{\xi_{discharge}} * \Delta{t}
        )
    $$
##### 4.1 *Additional* Constraints
* Limiting power flows BESS-to-Grid and PV-to-Grid in energy arbitrage
    $$
        P_{PV-to-Grid}[t] + P_{BESS-to-Grid}[t] <= PowerFlowLimit
    $$
* Limiting the BESS charging to either charge or discharge in the same hour
    $$
        \forall \quad t; \qquad (P_{PV-to-BESS}[t], P_{Grid-to-BESS}[t])*ChargingState\ \ AND\ \ (P_{BESS-to-Load}[t], P_{BESS-to-Grid}[t])*(1-ChargingState)
    $$
    * Subject to:
    * SoC tracking
    $$
        SoC[t] = (
            SoC[t-1]
            + ( P_{PV-to-BESS}[t] + P_{Grid-to-BESS}[t] * \xi_{charge} * ChargingState) * \Delta{t}
            - \frac{P_{BESS-to-Load}[t] + P_{BESS-to-Grid}[t]}{\xi_{discharge}} * (1-ChargingState) * \Delta{t}
        )
    $$

___
### Implementation notes
* General approach (for now) is to use the *ConcreteModel*
    * If *AbstractModel*, how could the model parameters,variables,etc be populated from the datafile OR the inputs of the model
    * *Why?* --> In the future, the model could be adapted to be compiled and to describe different kinds of and setups of energy systems with different technologies
* Be mindful of how the data is loaded and called in the model to avoid integration errors (like the previous error with numpy)
* Convert the model to kW (previously MW) to avoid issues/error with integer to float conversion
* *OBS*: SoC[*t*] needs a previous value, so we need to artificially override the initial SoC (or implement an SoC for *t* = -1)
* In future implementations/improvements, try converting the time-step to seconds/minutes.
    * *Reason* is to have a more realistic dispatch profile where BESS can charge and discharge in the same hour
* To keep in mind for validation
    * I could also run it with different curtailment penalties and see their effect
    * Underlying difference in optimisation problem when optimising only each time-step vs. optimising a time series window

In [ ]:
from pyomo.environ import *